In [1]:
import torch
from transformers import MarianTokenizer, MarianMTModel, AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, AutoConfig
from transformers.generation import BeamSearchScorer

d:\Users\niemi\Documents\Testing\Cuda\hf_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
term_model = MarianMTModel.from_pretrained("D:/Users/niemi/Documents/Testing/Cuda/termmodel.eng-fin")
term_tokenizer = MarianTokenizer.from_pretrained("D:/Users/niemi/Documents/Testing/Cuda/termmodel.eng-fin")

In [2]:
viking_vocab_model = MarianMTModel.from_pretrained("D:/Users/niemi/Documents/Testing/Cuda/converted-vocab_fix_model_viking7")
viking_tokenizer = AutoTokenizer.from_pretrained("LumiOpen/Viking-7B")

In [4]:
branch = "1000B"
viking_model = AutoModelForCausalLM.from_pretrained(
    "LumiOpen/Viking-7B",
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    revision=branch,
).to("cuda")

viking_tokenizer = AutoTokenizer.from_pretrained("LumiOpen/Viking-7B")

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  4.77it/s]


In [50]:
class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self,tokenizer, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]
        self.tokenizer = tokenizer
        self.batch_stop_list = []

    def __call__(self, batch_input_ids: torch.LongTensor, scores: torch.FloatTensor):
        source_id = 0
        for input_ids in batch_input_ids:
            if source_id in self.batch_stop_list:
                source_id += 1
                continue
            last_token = input_ids[-1]
            for stop in self.stops:
                if self.tokenizer.decode(stop) in self.tokenizer.decode(last_token):
                    print("stop:",self.tokenizer.decode(stop))
                    print("last token:",self.tokenizer.decode(last_token))
                    self.batch_stop_list.append(source_id)
            source_id += 1
        if len(self.batch_stop_list) == len(batch_input_ids):
            return True
        else:
            return False
        
# Stop at line break
stop_words = ["<|im_end|>"]
stop_words_ids = [viking_tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for stop_word in stop_words]

In [51]:
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(viking_tokenizer, stops=stop_words_ids)])
inputs = viking_tokenizer("<|im_start|>user\nTranslate into Finnish: The Sheilas’ Wheels owner, Esure, will be sold to the Belgian insurer Ageas.<|im_end|>\n<|im_start|>assistant\n",return_tensors="pt").to("cuda")
translated = viking_model.generate(**inputs,max_length=200, stopping_criteria=stopping_criteria)
print(viking_tokenizer.decode(translated[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


stop: <|im_end|>
last token: <|im_end|>
<|im_start|>user
Translate into Finnish: The Sheilas’ Wheels owner, Esure, will be sold to the Belgian insurer Ageas.<|im_end|>
<|im_start|>assistant
Sheilas' Wheelsin omistaja Esure myydään belgialaiselle vakuutusyhtiölle Ageasille.<|im_end|>


In [43]:
print(term_tokenizer.get_vocab()["augmentsymbol1"])

NameError: name 'term_tokenizer' is not defined

In [ ]:

def augment_tokenize(text, terms, tokenizer, device="cpu"):
    vocab = tokenizer.get_vocab()
    text_tokenized = tokenizer(text).input_ids
    for term_source,term_target in terms:
        term_source_tokenized = tokenizer(term_source).input_ids[:-1]
        term_target_tokenized = list(tokenizer(text_target=term_target).input_ids)[:-1]
        term_target_tokenized = [vocab["augmentsymbol1"]] + \
            term_target_tokenized + \
            [vocab["augmentsymbol2"]]
            
        current_aug_part_index = 0
        new_text_tokenized = []
        for token in text_tokenized:
            #TODO: add check for the word continuing
            if current_aug_part_index == len(term_source_tokenized):
                new_text_tokenized += [vocab["augmentsymbol0"]] + term_source_tokenized + \
                term_target_tokenized
                current_aug_part_index = 0
            if token == term_source_tokenized[current_aug_part_index]:
                current_aug_part_index += 1
            elif current_aug_part_index > 1:
                new_text_tokenized += term_source_tokenized[0:current_aug_part_index]
                new_text_tokenized.append(token)
                current_aug_part_index = 0
            else:
                new_text_tokenized.append(token)
        text_tokenized = new_text_tokenized

    input_ids = torch.tensor([text_tokenized], device=device)  # batch dimension added
    attention_mask = torch.ones_like(input_ids, device=device)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }

In [ ]:
terms = [("student","oppilas"),("test","koe")]
inputs = augment_tokenize("The student passed the test.", terms, term_tokenizer)
print(inputs["input_ids"])
print(term_tokenizer.decode(inputs["input_ids"][0]))
inputs1 = term_tokenizer("The student passed the test.",return_tensors="pt")
print(inputs1)
# Generate translation with max length
translated = term_model.generate(**inputs, max_length=100, num_beams=12)

# Decode the translation
translated_text = term_tokenizer.decode(translated[0], skip_special_tokens=True)
print(translated_text)

tensor([[57829, 57829, 48685, 57440, 45603, 57432, 34377, 57291, 36046, 48683,
         57440, 48558, 57432, 24547, 57291,    46, 41756]])
<pad> <pad> Theaugmentsymbol0 studentaugmentsymbol1 oppilasaugmentsymbol2 passed theaugmentsymbol0 testaugmentsymbol1 koeaugmentsymbol2.</s>
{'input_ids': tensor([[48685,  6090, 16089, 30124, 55986, 28752,  7250,   275, 45603,  6126,
         41755, 55986, 28752,  7250,   446, 34235, 36996,  5567, 51683, 30124,
         55986, 28752,  7250,  1177, 36046, 48683, 48558,    46, 41756]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]])}
Testin läpäisi auppsymbol1 oppilaseugmentsymbol2 läpäistyään tutkimuksensa, jonka tulokset olivat samat kuin muidenkin opiskelijoiden ja opiskelijoiden, joiden tutkimustulokset olivat samat kuin opiskelijoiden ja opiskelijoiden, joiden tutkimustulokset olivat samat kuin opiskelijoiden ja opiskelijoiden, joiden tutkimustulokset olivat samat kuin 

In [2]:
def load_model(model_name_or_path, device):
    config = AutoConfig.from_pretrained(model_name_or_path)

    # Check model type by architecture string
    if "Marian" in config.model_type or config.architectures and any("Marian" in arch for arch in config.architectures):
        model = MarianMTModel.from_pretrained(model_name_or_path).to(device).eval()
        model_type = "marian"
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_name_or_path,
            torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16).to(device).eval()
        model_type = "causal_lm"

    return model, model_type

In [3]:
def load_tokenizer(model_name_or_path):
    config = AutoConfig.from_pretrained(model_name_or_path)

    # Check model type by architecture string
    if "Marian" in config.model_type or config.architectures and any("Marian" in arch for arch in config.architectures):
        tokenizer = MarianTokenizer.from_pretrained(model_name_or_path)
    else:
        tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

    return tokenizer

In [4]:
class ShallowFusion():
    def __init__(self, models_info):
        self.models_info = models_info
        # === Load tokenizer and models ===
        # The tokenizer of the first model is used
        self.tokenizer = load_tokenizer(models_info[0]["name"])
        
        # tokenizer = AutoTokenizer.from_pretrained("LumiOpen/Viking-7B")
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.init_models()

    def init_models(self):
        self.models = []        
        for info in self.models_info:
            model, model_type = load_model(info["name"], self.device)
            self.models.append((model,model_type))

    def initialize_inputs(self, src_sentences_per_model):
        self.encoder_input_ids_list = []
        self.attention_masks_list = []
        self.decoder_input_ids_list = []

        num_beams = 4
        self.eos_token_id = self.tokenizer.eos_token_id
        self.pad_token_id = self.tokenizer.pad_token_id  # Marian uses pad token to start decoding
        self.start_token_id = self.tokenizer.pad_token_id  # Marian uses pad token to start decoding

        for (sentence,terms),(model,model_type) in zip(src_sentences_per_model,self.models):
            # Tokenize and prepare inputs
            if terms:
                enc_inputs = augment_tokenize(sentence, terms, self.tokenizer, self.device)
            else:
                enc_inputs = self.tokenizer(sentence, return_tensors="pt").to(self.device)
                
            encoder_input_ids = enc_inputs["input_ids"].expand(num_beams, -1).clone()
            attention_mask = enc_inputs["attention_mask"].expand(num_beams, -1).clone()

            self.encoder_input_ids_list.append(encoder_input_ids)
            self.attention_masks_list.append(attention_mask)

            if model_type == "marian":
                # Init decoder_input_ids with a start token
                decoder_input_ids = torch.full(
                    (num_beams, 1),
                    fill_value=self.start_token_id,
                    dtype=torch.long,
                    device=self.device,
                )
            else:
                # LLMs keep track of decoder input ids, but concat them with the input for generation
                decoder_input_ids = torch.empty((num_beams, 0), dtype=torch.long, device=self.device)
            self.decoder_input_ids_list.append(decoder_input_ids)
    
    def translate(self, src_sentences_per_model, num_beams=4, max_length=50):
        self.initialize_inputs(src_sentences_per_model)
        
        # === Beam search setup ===
        beam_scorer = BeamSearchScorer(
            batch_size=1,
            num_beams=num_beams,
            device=self.device,
            length_penalty=1.0,
            do_early_stopping=True,
            num_beam_hyps_to_keep=num_beams,
        )

        beam_scores = torch.zeros((num_beams,), dtype=torch.float, device=self.device)
        beam_scores[1:] = -1e9  # Only first beam is active at the beginning

        cur_len = 1  # decoder_input_ids starts with 1 token

        # === Step-by-step beam search loop ===
        while cur_len < max_length:
            all_log_probs = []

            # === Each model provides logits from its own source + decoder input ===
            for (model,model_type), encoder_input_ids, attention_mask, decoder_input_ids in zip(
                self.models, self.encoder_input_ids_list, self.attention_masks_list, self.decoder_input_ids_list
            ):
                with torch.no_grad():
                    if model_type == "marian":
                        outputs = model(
                            input_ids=encoder_input_ids,
                            attention_mask=attention_mask,
                            decoder_input_ids=decoder_input_ids,
                        )
                        logits = outputs.logits[:, -1, :]  # (num_beams, vocab_size)
                        logits[:, [self.pad_token_id]] = float("-inf")
                    else:
                        input_ids = torch.cat([encoder_input_ids, decoder_input_ids], dim=-1)
                        outputs = model(
                            input_ids=input_ids,
                            attention_mask=attention_mask,
                        )
                        logits = outputs.logits[:, -1, :]  # (num_beams, vocab_size)

                    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
                    all_log_probs.append(log_probs)

            # === Shallow fusion: average log_probs ===
            avg_log_probs = torch.stack(all_log_probs).mean(dim=0)

            # === Beam step ===
            next_beam_scores, next_tokens = torch.topk(avg_log_probs, 2, dim=1)
            next_beam_scores += beam_scores[:, None]

            next_beam_scores = next_beam_scores.view(1, -1)
            next_tokens = next_tokens.view(1, -1)
            next_indices = torch.arange(num_beams, device=self.device).repeat_interleave(2).view(1, -1)

            decoder_input_ids_for_process = self.decoder_input_ids_list[0]
            
            beam_outputs = beam_scorer.process(
                decoder_input_ids_for_process,
                next_beam_scores,
                next_tokens,
                next_indices,
                eos_token_id=self.eos_token_id,
            )

            # === Update decoder_input_ids and beam scores ===
            for i in range(len(self.models)):
                self.decoder_input_ids_list[i] = torch.cat(
                    [
                        self.decoder_input_ids_list[i][beam_outputs.data["next_beam_indices"]],
                        beam_outputs.data["next_beam_tokens"].unsqueeze(-1),
                    ],
                    dim=-1,
                )

            beam_scores = beam_outputs.data["next_beam_scores"]
            cur_len += 1

            if beam_scorer.is_done:
                break

        # === Finalize hypotheses ===
        final_outputs = beam_scorer.finalize(
            self.decoder_input_ids_list[0],
            beam_scores,
            final_beam_tokens=None,
            final_beam_indices=None,
            max_length=cur_len,
            pad_token_id=self.pad_token_id,
            eos_token_id=self.eos_token_id
        )

        # === Decode translations ===
        translation = [self.tokenizer.decode(t, skip_special_tokens=False) for t in final_outputs.data["sequences"][0]]
        translations = self.tokenizer.batch_decode(final_outputs.data["sequences"], skip_special_tokens=True,)
        return translations


In [1]:
# === Define models and their unique source inputs ===
models_info = [
    {
        "name": "LumiOpen/Viking-7B"
    },
    {
        "name": "../converted-vocab_fix_model_viking7",
    },
]

shallow_fusion = ShallowFusion(models_info)

NameError: name 'ShallowFusion' is not defined

In [11]:
def viking_template(sentence):
    return (f"<|im_start|>user\nTranslate into Finnish: {sentence}.<|im_end|>\n<|im_start|>assistant\n",[])

def marian_llmvoc_template(sentence):
    return (sentence + "</s>", [])

In [14]:
test_sent = " The colourless and odourless gas is used to create an atmosphere of extreme subzero temperatures."
input_sentences = [viking_template(test_sent), marian_llmvoc_template(test_sent)]
print(input_sentences)
translations = shallow_fusion.translate(input_sentences)

print("\n🌍 Shallow Fusion Translations (Multi-Input):")
print("\n".join(translations))

[('<|im_start|>user\nTranslate into Finnish:  The colourless and odourless gas is used to create an atmosphere of extreme subzero temperatures..<|im_end|>\n<|im_start|>assistant\n', []), (' The colourless and odourless gas is used to create an atmosphere of extreme subzero temperatures.</s>', [])]

🌍 Shallow Fusion Translations (Multi-Input):
Väritön ja hajuton kaasu on käytetty luomaan äärimmäisen pakkasilman ilmapiiri.
Väritön ja hajuton kaasu on käytetty luomaan äärimmäisen pakkasilman.
Väritön ja hajuton kaasu on käytetty luomaan äärimmäisen pakkasilman.

Väritön ja hajuton kaasu on käytetty luomaan äärimmäisen pakkasilman ilmapiiri.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("LumiOpen/Viking-7B")
#tokenizer = MarianTokenizer.from_pretrained("/home/tommi/hf_testing/converted-vocab_fix_model")
model = MarianMTModel.from_pretrained("../converted-vocab_fix_model_viking7").to("cpu")
# THIS IS THE CORRECT WAY: eos token at the end of input, pad at start
input_ids = tokenizer("This is a test.</s>",return_tensors="pt").input_ids.to("cpu")
decoder_input_ids = tokenizer("<pad>",return_tensors="pt").input_ids.to("cpu")
for _ in range(100):
    with torch.no_grad():
        outputs = model(
        input_ids=input_ids,
        decoder_input_ids=decoder_input_ids,
    )
        logits = outputs.logits
        next_token_logits = logits[:, -1, :]
        next_token_logits[0][tokenizer.pad_token_id] = float("-inf")
        next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
        decoder_input_ids = torch.cat([decoder_input_ids, next_token_id], dim=-1)
        if next_token_id[0][0] == tokenizer.eos_token_id:
            break
        new_token = tokenizer.decode(next_token_id.squeeze(), skip_special_tokens=True)
        print(new_token, end="", flush=True)
    

 Tämä on testi.